## Introduction
In this competition, due to the extremely large image size, 
an efficient way of removing background (=making tiles) is very important.

This implementation is base on [GM Iafoss](https://www.kaggle.com/iafoss)'s [notebook](https://www.kaggle.com/code/iafoss/panda-16x128x128-tiles) from PANDA competition.

In [ ]:
from pathlib import Path
import numpy as np
import skimage.io as io
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
image_paths = list(Path('../input/mayo-clinic-strip-ai/test/').glob('*.tif'))
image_paths

In [ ]:
def load_image(img_path, resize_factor=16):
    img = io.imread(img_path)
    img = cv2.resize(img, dsize=None, fx=1/resize_factor, fy=1/resize_factor)
    return img

In [ ]:
def make_tiles(img, tile_size=256, num_tiles=4):
    '''
    img: np.ndarray with dtype np.uint8 and shape (width, height, channel)
    '''
    w, h, ch = img.shape
    pad0, pad1 = (tile_size - w%tile_size) % tile_size, (tile_size - h%tile_size) % tile_size
    padding = [[pad0//2, pad0-pad0//2], [pad1//2, pad1-pad1//2], [0, 0]]
    img = np.pad(img, padding, mode='constant', constant_values=255)
    img = img.reshape(img.shape[0]//tile_size, tile_size, img.shape[1]//tile_size, tile_size, ch)
    img = img.transpose(0, 2, 1, 3, 4).reshape(-1, tile_size, tile_size, ch)
    if len(img) < num_tiles: # pad images so that the output shape be the same
        padding = [[0, num_tiles-len(img)], [0, 0], [0, 0], [0, 0]]
        img = np.pad(img, padding, mode='constant', constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0], -1).sum(-1))[:num_tiles] # pick up Top N dark tiles
    img = img[idxs]
    return img

In [ ]:
image = load_image(image_paths[0])
plt.imshow(image)
plt.show()

In [ ]:
tiles = make_tiles(image)
tiles.shape

In [ ]:
for tile in tiles:
    plt.imshow(tile)
    plt.show()

## Benchmark
**It takes only around 20s to process 500 images!**

In [ ]:
%%time
for _ in tqdm(range(500)):
    _ = make_tiles(image)

In [ ]:
nan